## Initialize Simulation Environment

In [1]:
#!/usr/bin/env python

"""demo_simulate_nyu_finger_double

Simple demo showing how the simulation setup works.

License: BSD 3-Clause License
Copyright (C) 2018-2021, New York University , Max Planck Gesellschaft
Copyright note valid unless otherwise stated in individual files.
All rights reserved.
"""

import time
from robot_properties_nyu_finger.config import NYUFingerDoubleConfig0, NYUFingerDoubleConfig1
from robot_properties_nyu_finger.wrapper import NYUFingerRobot
from bullet_utils.env import BulletEnv
import numpy as np
import pinocchio as pin

pybullet build time: Aug  2 2021 16:06:49


In [2]:
# ! Create a Pybullet simulation environment before any robots !
env = BulletEnv()

# Create a robot instance. This adds the robot to the simulator as well.
config0 = NYUFingerDoubleConfig0()
config1 = NYUFingerDoubleConfig1()
finger0 = NYUFingerRobot(config=config0)
finger1 = NYUFingerRobot(config=config1)

# pinocchio robot wrapper
pin_robot_0 = config0.buildRobotWrapper()
pin_robot_1 = config1.buildRobotWrapper()

# Add the robot to the env to update the internal structure of the robot
# at every simulation steps.
env.add_robot(finger0)
env.add_robot(finger1)

## Define Controller /s

In [3]:
import numpy as np
import matplotlib.pylab as plt

from dynamic_graph_head import ThreadHead, SimHead

HoldPDController 

In [4]:
from dynamic_graph_head import HoldPDController

Zero Torque Controller

In [5]:
class ZeroTorquesController:
    def __init__(self, head):
        self.head = head

        # Zero the commands.
        self.tau = np.zeros_like(head.get_sensor("joint_positions"))

    def warmup(self, thread_head):
        pass

    def run(self, thread_head):
        self.head.set_control("ctrl_joint_torques", self.tau)

Gravity Compensation Controller

In [6]:
class GravityCompensationController:
    def __init__(self, head):
        """
        Args:
            head: Instance of DGHead or SimHead.
        """
        self.head = head
        
        self.joint_positions = head.get_sensor("joint_positions")
        self.joint_velocities = head.get_sensor("joint_velocities")
        
        self.zeros_pos = np.zeros_like(self.joint_positions)
        # TODO: get the pinochhio wrapper in a better way
        self.robot = head._robot.pin_robot # not the best way, but welp it works

    def warmup(self, thread_head):
        self.base_pos = self.joint_positions.copy()
        return

    def rnea(self, q, dq, ddq):
        return pin.rnea(self.robot.model, self.robot.data, q, dq, ddq)
    
    def run(self, thread_head):
        self.tau = self.rnea(self.joint_positions, self.zeros_pos, self.zeros_pos)
        # print(self.tau)
        self.head.set_control("ctrl_joint_torques", self.tau)
        return

## Initialize robot with defined controller

In [7]:
head0 = SimHead(finger0, with_sliders=False) # left
head1 = SimHead(finger1, with_sliders=False) # right

hold_ctrl0 = HoldPDController(head0, 3., 0.05, False)  
hold_ctrl1 = HoldPDController(head1, 3., 0.05, False)

zero_ctrl0 = ZeroTorquesController(head0)
zero_ctrl1 = ZeroTorquesController(head1)

grav_ctrl0 = GravityCompensationController(head0)
grav_ctrl1 = GravityCompensationController(head1)

In [8]:
thread_head = ThreadHead(
    0.001, # dt.
    [hold_ctrl0, hold_ctrl1], # Safety controllers.
    {'head0': head0, 'head1':head1}, # Heads to read / write from.
    [], 
    env # Environment to step.
)

In [9]:
env.step(1000)

## Run Simulation

In [13]:
thread_head.switch_controllers([grav_ctrl0, grav_ctrl1])

[ 6.55952308e-02  5.17419211e-17 -3.71169151e-18]
[ 6.55952308e-02  5.17419211e-17 -3.71169151e-18]


In [14]:
for i in range(1000):
    thread_head.run_main_loop(True)

[ 6.55920817e-02  5.17921141e-17 -3.71528981e-18]
[ 6.55920817e-02  5.17921141e-17 -3.71528981e-18]
[ 6.55889327e-02  5.18423545e-17 -3.71889151e-18]
[ 6.55889327e-02  5.18423545e-17 -3.71889151e-18]
[ 6.55857838e-02  5.18926524e-17 -3.72249732e-18]
[ 6.55857838e-02  5.18926524e-17 -3.72249732e-18]
[ 6.55826350e-02  5.19430237e-17 -3.72610841e-18]
[ 6.55826350e-02  5.19430237e-17 -3.72610841e-18]
[ 6.55794863e-02  5.19934786e-17 -3.72972549e-18]
[ 6.55794863e-02  5.19934786e-17 -3.72972549e-18]
[ 6.55763378e-02  5.20439974e-17 -3.73334715e-18]
[ 6.55763378e-02  5.20439974e-17 -3.73334715e-18]
[ 6.55731893e-02  5.20945961e-17 -3.73697454e-18]
[ 6.55731893e-02  5.20945961e-17 -3.73697454e-18]
[ 6.55700410e-02  5.21452433e-17 -3.74060541e-18]
[ 6.55700410e-02  5.21452433e-17 -3.74060541e-18]
[ 6.55668928e-02  5.21959432e-17 -3.74424004e-18]
[ 6.55668928e-02  5.21959432e-17 -3.74424004e-18]
[ 6.55637447e-02  5.22466998e-17 -3.74787875e-18]
[ 6.55637447e-02  5.22466998e-17 -3.74787875e-18]


[ 6.53342396e-02  5.60569299e-17 -4.02102345e-18]
[ 6.53310999e-02  5.61097572e-17 -4.02481028e-18]
[ 6.53310999e-02  5.61097572e-17 -4.02481028e-18]
[ 6.53279603e-02  5.61626061e-17 -4.02859865e-18]
[ 6.53279603e-02  5.61626061e-17 -4.02859865e-18]
[ 6.53248208e-02  5.62154511e-17 -4.03238674e-18]
[ 6.53248208e-02  5.62154511e-17 -4.03238674e-18]
[ 6.53216814e-02  5.62683027e-17 -4.03617529e-18]
[ 6.53216814e-02  5.62683027e-17 -4.03617529e-18]
[ 6.53185422e-02  5.63211533e-17 -4.03996376e-18]
[ 6.53185422e-02  5.63211533e-17 -4.03996376e-18]
[ 6.53154030e-02  5.63740190e-17 -4.04375331e-18]
[ 6.53154030e-02  5.63740190e-17 -4.04375331e-18]
[ 6.53122640e-02  5.64268804e-17 -4.04754254e-18]
[ 6.53122640e-02  5.64268804e-17 -4.04754254e-18]
[ 6.53091251e-02  5.64797598e-17 -4.05133305e-18]
[ 6.53091251e-02  5.64797598e-17 -4.05133305e-18]
[ 6.53059863e-02  5.65326495e-17 -4.05512429e-18]
[ 6.53059863e-02  5.65326495e-17 -4.05512429e-18]
[ 6.53028476e-02  5.65855361e-17 -4.05891531e-18]


[ 6.50521189e-02  6.08453424e-17 -4.36424377e-18]
[ 6.50521189e-02  6.08453424e-17 -4.36424377e-18]
[ 6.50489894e-02  6.08990682e-17 -4.36809438e-18]
[ 6.50489894e-02  6.08990682e-17 -4.36809438e-18]
[ 6.50458600e-02  6.09527988e-17 -4.37194533e-18]
[ 6.50458600e-02  6.09527988e-17 -4.37194533e-18]
[ 6.50427307e-02  6.10065386e-17 -4.37579693e-18]
[ 6.50427307e-02  6.10065386e-17 -4.37579693e-18]
[ 6.50396015e-02  6.10602920e-17 -4.37964949e-18]
[ 6.50396015e-02  6.10602920e-17 -4.37964949e-18]
[ 6.50364724e-02  6.11140515e-17 -4.38350249e-18]
[ 6.50364724e-02  6.11140515e-17 -4.38350249e-18]
[ 6.50333434e-02  6.11678216e-17 -4.38735624e-18]
[ 6.50333434e-02  6.11678216e-17 -4.38735624e-18]
[ 6.50302146e-02  6.12215947e-17 -4.39121020e-18]
[ 6.50302146e-02  6.12215947e-17 -4.39121020e-18]
[ 6.50270858e-02  6.12753872e-17 -4.39506555e-18]
[ 6.50270858e-02  6.12753872e-17 -4.39506555e-18]
[ 6.50239572e-02  6.13291917e-17 -4.39892174e-18]
[ 6.50239572e-02  6.13291917e-17 -4.39892174e-18]


[ 6.47771527e-02  6.56226803e-17 -4.70661649e-18]
[ 6.47771527e-02  6.56226803e-17 -4.70661649e-18]
[ 6.47740331e-02  6.56780071e-17 -4.71058129e-18]
[ 6.47740331e-02  6.56780071e-17 -4.71058129e-18]
[ 6.47709136e-02  6.57333863e-17 -4.71454985e-18]
[ 6.47709136e-02  6.57333863e-17 -4.71454985e-18]
[ 6.47677942e-02  6.57887868e-17 -4.71851994e-18]
[ 6.47677942e-02  6.57887868e-17 -4.71851994e-18]
[ 6.47646750e-02  6.58442252e-17 -4.72249273e-18]
[ 6.47646750e-02  6.58442252e-17 -4.72249273e-18]
[ 6.47615558e-02  6.58996821e-17 -4.72646684e-18]
[ 6.47615558e-02  6.58996821e-17 -4.72646684e-18]
[ 6.47584368e-02  6.59551622e-17 -4.73044261e-18]
[ 6.47584368e-02  6.59551622e-17 -4.73044261e-18]
[ 6.47553179e-02  6.60106582e-17 -4.73441951e-18]
[ 6.47553179e-02  6.60106582e-17 -4.73441951e-18]
[ 6.47521991e-02  6.60661745e-17 -4.73839787e-18]
[ 6.47521991e-02  6.60661745e-17 -4.73839787e-18]
[ 6.47490804e-02  6.61217157e-17 -4.74237800e-18]
[ 6.47490804e-02  6.61217157e-17 -4.74237800e-18]


[ 6.44937309e-02  7.07767926e-17 -5.07594931e-18]
[ 6.44937309e-02  7.07767926e-17 -5.07594931e-18]
[ 6.44906215e-02  7.08349499e-17 -5.08011655e-18]
[ 6.44906215e-02  7.08349499e-17 -5.08011655e-18]
[ 6.44875123e-02  7.08931196e-17 -5.08428466e-18]
[ 6.44875123e-02  7.08931196e-17 -5.08428466e-18]
[ 6.44844032e-02  7.09513184e-17 -5.08845485e-18]
[ 6.44844032e-02  7.09513184e-17 -5.08845485e-18]
[ 6.44812942e-02  7.10095628e-17 -5.09262831e-18]
[ 6.44812942e-02  7.10095628e-17 -5.09262831e-18]
[ 6.44781853e-02  7.10678458e-17 -5.09680453e-18]
[ 6.44781853e-02  7.10678458e-17 -5.09680453e-18]
[ 6.44750765e-02  7.11261603e-17 -5.10098300e-18]
[ 6.44750765e-02  7.11261603e-17 -5.10098300e-18]
[ 6.44719678e-02  7.11844874e-17 -5.10516236e-18]
[ 6.44719678e-02  7.11844874e-17 -5.10516236e-18]
[ 6.44688593e-02  7.12428436e-17 -5.10934381e-18]
[ 6.44688593e-02  7.12428436e-17 -5.10934381e-18]
[ 6.44657508e-02  7.13012338e-17 -5.11352768e-18]
[ 6.44657508e-02  7.13012338e-17 -5.11352768e-18]


[ 6.39852994e-02  8.08285974e-17 -5.79615548e-18]
[ 6.39852994e-02  8.08285974e-17 -5.79615548e-18]
[ 6.39822085e-02  8.08933762e-17 -5.80079657e-18]
[ 6.39822085e-02  8.08933762e-17 -5.80079657e-18]
[ 6.39791177e-02  8.09581971e-17 -5.80544066e-18]
[ 6.39791177e-02  8.09581971e-17 -5.80544066e-18]
[ 6.39760269e-02  8.10230889e-17 -5.81008984e-18]
[ 6.39760269e-02  8.10230889e-17 -5.81008984e-18]
[ 6.39729363e-02  8.10880332e-17 -5.81474278e-18]
[ 6.39729363e-02  8.10880332e-17 -5.81474278e-18]
[ 6.39698458e-02  8.11529998e-17 -5.81939732e-18]
[ 6.39698458e-02  8.11529998e-17 -5.81939732e-18]
[ 6.39667555e-02  8.12179941e-17 -5.82405382e-18]
[ 6.39667555e-02  8.12179941e-17 -5.82405382e-18]
[ 6.39636652e-02  8.12830331e-17 -5.82871353e-18]
[ 6.39636652e-02  8.12830331e-17 -5.82871353e-18]
[ 6.39605750e-02  8.13481223e-17 -5.83337683e-18]
[ 6.39605750e-02  8.13481223e-17 -5.83337683e-18]
[ 6.39574850e-02  8.14132551e-17 -5.83804326e-18]
[ 6.39574850e-02  8.14132551e-17 -5.83804326e-18]


[ 6.37106442e-02  8.67814293e-17 -6.22263564e-18]
[ 6.37075632e-02  8.68505360e-17 -6.22758655e-18]
[ 6.37075632e-02  8.68505360e-17 -6.22758655e-18]
[ 6.37044823e-02  8.69197104e-17 -6.23254229e-18]
[ 6.37044823e-02  8.69197104e-17 -6.23254229e-18]
[ 6.37014016e-02  8.69889223e-17 -6.23750073e-18]
[ 6.37014016e-02  8.69889223e-17 -6.23750073e-18]
[ 6.36983209e-02  8.70581774e-17 -6.24246226e-18]
[ 6.36983209e-02  8.70581774e-17 -6.24246226e-18]
[ 6.36952404e-02  8.71274934e-17 -6.24742814e-18]
[ 6.36952404e-02  8.71274934e-17 -6.24742814e-18]
[ 6.36921599e-02  8.71968640e-17 -6.25239794e-18]
[ 6.36921599e-02  8.71968640e-17 -6.25239794e-18]
[ 6.36890796e-02  8.72663187e-17 -6.25737376e-18]
[ 6.36890796e-02  8.72663187e-17 -6.25737376e-18]
[ 6.36859994e-02  8.73358038e-17 -6.26235175e-18]
[ 6.36859994e-02  8.73358038e-17 -6.26235175e-18]
[ 6.36829193e-02  8.74053368e-17 -6.26733318e-18]
[ 6.36829193e-02  8.74053368e-17 -6.26733318e-18]
[ 6.36798394e-02  8.74749235e-17 -6.27231845e-18]


[ 6.32007292e-02  9.85135108e-17 -7.06302115e-18]
[ 6.32007292e-02  9.85135108e-17 -7.06302115e-18]
[ 6.31976667e-02  9.85842664e-17 -7.06808846e-18]
[ 6.31976667e-02  9.85842664e-17 -7.06808846e-18]
[ 6.31946044e-02  9.86550195e-17 -7.07315559e-18]
[ 6.31946044e-02  9.86550195e-17 -7.07315559e-18]
[ 6.31915421e-02  9.87257404e-17 -7.07822039e-18]
[ 6.31915421e-02  9.87257404e-17 -7.07822039e-18]
[ 6.31884800e-02  9.87964467e-17 -7.08328414e-18]
[ 6.31884800e-02  9.87964467e-17 -7.08328414e-18]
[ 6.31854180e-02  9.88671561e-17 -7.08834809e-18]
[ 6.31854180e-02  9.88671561e-17 -7.08834809e-18]
[ 6.31823560e-02  9.89378625e-17 -7.09341181e-18]
[ 6.31823560e-02  9.89378625e-17 -7.09341181e-18]
[ 6.31792942e-02  9.90085717e-17 -7.09847572e-18]
[ 6.31792942e-02  9.90085717e-17 -7.09847572e-18]
[ 6.31762325e-02  9.90792778e-17 -7.10353939e-18]
[ 6.31762325e-02  9.90792778e-17 -7.10353939e-18]
[ 6.31731709e-02  9.91499863e-17 -7.10860323e-18]
[ 6.31731709e-02  9.91499863e-17 -7.10860323e-18]


[ 6.29255517e-02  1.04897615e-16 -7.52018508e-18]
[ 6.29255517e-02  1.04897615e-16 -7.52018508e-18]
[ 6.29224993e-02  1.04968856e-16 -7.52528609e-18]
[ 6.29224993e-02  1.04968856e-16 -7.52528609e-18]
[ 6.29194469e-02  1.05040120e-16 -7.53038873e-18]
[ 6.29194469e-02  1.05040120e-16 -7.53038873e-18]
[ 6.29163947e-02  1.05111400e-16 -7.53549257e-18]
[ 6.29163947e-02  1.05111400e-16 -7.53549257e-18]
[ 6.29133425e-02  1.05182704e-16 -7.54059803e-18]
[ 6.29133425e-02  1.05182704e-16 -7.54059803e-18]
[ 6.29102905e-02  1.05254000e-16 -7.54570297e-18]
[ 6.29102905e-02  1.05254000e-16 -7.54570297e-18]
[ 6.29072386e-02  1.05325307e-16 -7.55080867e-18]
[ 6.29072386e-02  1.05325307e-16 -7.55080867e-18]
[ 6.29041868e-02  1.05396619e-16 -7.55591469e-18]
[ 6.29041868e-02  1.05396619e-16 -7.55591469e-18]
[ 6.29011351e-02  1.05467941e-16 -7.56102145e-18]
[ 6.29011351e-02  1.05467941e-16 -7.56102145e-18]
[ 6.28980835e-02  1.05539268e-16 -7.56612852e-18]
[ 6.28980835e-02  1.05539268e-16 -7.56612852e-18]
